In [1]:
import os
import sys
from glob import glob

module_path = os.path.abspath(os.path.join('../../'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import desispec
from desispec.io import read_spectra, write_spectra
from desispec.spectra import Spectra

from desiutil.log import get_logger, DEBUG

from desidiff.src.group_tiles import *
from desidiff.src.dates_to_process import *
from desidiff.src.coadd import *
from desidiff.src.scores import *
from desidiff.src.ContinuumFitFilter_desidiff import *

from timedomain.sp_utils import SkyPortal as sp
import requests
import datetime
import heapq
import time

import numpy
from astropy.time import Time
from astropy.table import Table, vstack, unique, SortedArray

%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from IPython import display

In [15]:
# read in and store in one place all the fibermap information in the spectra files

night = 20210608


for filename in glob(f"/global/project/projectdirs/desi/spectro/redux/daily/tiles/cumulative/*/{night}/spectra-*.fits"):
    print(filename)
    
    tile = filename.split('/')[-3]
    petal = filename.split('-')[1]
    current_spectra = read_spectra(filename)
    
    ### select only spectra from that night
    current_spectra = desispec.coaddition.coadd_cameras(current_spectra.select(nights = night))
    
    t = Table.read(filename, format='fits',hdu=1, memmap=True) 
    ##### DAVE'S ADDITION ##############
    targetcols = [i for i in t.colnames if i[-7:] =='_TARGET']
    nonzerocheck = [True in k for k in [[j != 0 for j in t[targetcols][i]] for i in range(len(t))]]
    #a really ugly line, basically generates a list of bools, 
    #True if there is at least one nonzero element in all columns ending in _TARGET
    t.remove_rows([i for i, val in enumerate(nonzerocheck) if not val])
    #This gets the index of all False values from the previous list and removes those rows
    t = t['TARGETID','TARGET_RA','TARGET_DEC','TILEID','OBJTYPE','PETAL_LOC','FIBERSTATUS','NIGHT']
    ######## END DAVE'S ADDITION ############
    t=t[numpy.logical_and(t['OBJTYPE']=='TGT', t['FIBERSTATUS']==0)]
   

    for filename in glob(f"/global/cfs/cdirs/desi/spectro/redux/*/tiles/cumulative/{tile}/*/coadd-{petal}-{tile}*.fits"):
        print(filename)
        try:
            if filename.split('/')[-6] == 'fuji' or filename.split('/')[-6] == 'guadalupe':
                ref_spectra = desispec.coaddition.coadd_cameras(read_spectra(filename))
                print('ref_spectra from {}'.format(filename))
            
        except:
            pass

        ### rr = table.read
        ### insert loop around targetids from daily: compare with ref_database

        
        # renormalize spectra to match each other
        # There is a significant background of spectra that have the same shape but different fluxes
        # This seems to be related to mistaken coordinates of bright sources
        norm = normalization(current_spectra.flux, current_spectra.mask, ref_spectra.flux, ref_spectra.mask)

        for key in (current_spectra.flux).keys():
            current_spectra.flux[key]=current_spectra.flux[key]/norm
            current_spectra.ivar[key]=current_spectra.ivar[key]*norm**2        

        diff_flux = {key: current_spectra.flux[key] - ref_spectra.flux[key]
                       for key in current_spectra.flux.keys()}
        diff_ivar = {key: 1./(1./current_spectra.ivar[key] + 1./ref_spectra.ivar[key])
                       for key in current_spectra.ivar.keys()}
        diff_mask = {key: current_spectra.mask[key] + ref_spectra.mask[key]
                       for key in current_spectra.mask.keys()}
        diff_wave = dict(current_spectra.wave)

        # Mask systematically problematic areas of the spectrum
        # trim red edge 
        if 'z' in diff_flux.keys():
            diff_mask['z'][diff_wave['z'] > lmaxz]=1

        # trim blue edge 
        if 'b' in diff_flux.keys():
            diff_mask['b'][diff_wave['b'] < lminb]=1

        if 'r' in diff_flux.keys():
            diff_mask['r'][diff_wave['r'] < lminr]=1
            diff_mask['r'][diff_wave['r'] > lmaxr]=1

        # mean-subtracted difference
        diff_flux_clipmean = clipmean(diff_flux,diff_ivar,diff_mask)

        ## filters 

        #difference spectrum may have broadband signal
        perband_filter = perband_SN(diff_flux_clipmean,diff_ivar,diff_mask)

        #fractional increase
        perband_inc = perband_increase(diff_flux_clipmean,diff_ivar,diff_mask, ref_flux,ref_ivar,ref_mask)

        # difference spectrum may have high-frequency signal
        perres_filter_broad = perconv_SN(diff_wave, diff_flux_clipmean,diff_ivar,diff_mask, ncon = 100, nsig = 17)

        perres_filter_narrow = perconv_SN(diff_wave, diff_flux_clipmean,diff_ivar,diff_mask, ncon = 5, nsig = 10)

        # Search for signature lines of TDEs, only interested in Galaxies
        linetable = line_finder(diff_wave, diff_flux_clipmean,diff_ivar,diff_mask,zinfo['Z'][0])
        # print(night, linetable)
        spectype = zinfo['SPECTYPE'][0]
        Hline_score = Hline_filter(linetable)
        # deriv_score = deriv_filter(diff_flux_clipmean, diff_ivar,diff_mask)

        #broadband
        bblogic = any(numpy.logical_and(numpy.array(list(perband_filter.values()))>10, numpy.array(list(perband_inc.values()))>0.25))
        narrowlinelogic = perres_filter_narrow >=2
        broadlinelogic = perres_filter_broad >=3
        # TDElogic = any([TDE_score == 2, TDE_score == 3, TDE_score == 4, TDE_score == 5])
        Hlinelogic = any([Hline_score >= 1])
        # derivlogic = any([deriv_score >= 3])

        logic = [bblogic, narrowlinelogic,  broadlinelogic, Hlinelogic]
        logic_name = ['Broadband', 'narrow line', 'broad line','Hline'] #must be in same order as logic!, use as mask
        logic_name = numpy.ma.masked_array(logic_name, mask = [not i for i in logic])
        plt.clf()
        if any(logic):
            #Uncomment next line if you want to print only those TargetIds that get plotted
            processed(t, tile, night)
            fig = plt.figure()
            for b in diff_flux.keys():
                w=numpy.where(diff_mask[b] ==0)[0]
                if b == list(diff_flux.keys())[-1]:
                    plt.plot(diff_wave[b][w],diff_flux[b][w],color='red', label = 'Difference')
                    plt.plot(current_wave[b][w],current_flux[b][w],color='blue',alpha=0.5, label = 'Current Spectrum')
                    plt.plot(ref_wave[b][w],ref_flux[b][w],color='green',alpha=0.5, label = 'Reference Spectrum')
                else:
                    plt.plot(diff_wave[b][w],diff_flux[b][w],color='red')
                    plt.plot(current_wave[b][w],current_flux[b][w],color='blue',alpha=0.5)
                    plt.plot(ref_wave[b][w],ref_flux[b][w],color='green',alpha=0.5)
            plt.legend()
            plt.xlim((lminb,lmaxz))
            plt.xlabel('Wavelength (A)')
            plt.ylabel('Flux') 
            plt.title(str(t) + "  " + str(night) + "  " + str(tile)  + "  " + str(logic_name))
            plt.savefig(pdf, format = 'pdf')
            plt.savefig(pdf1, format = 'pdf')
            plt.close()
            # end desidiff.ipynb code

            #SkyPortal functionality begins:

            #SkyPortal's Id for the object
            objID = 'DESI{}'.format(str(t))

            #Code to check if this object already exists in SkyPortal. If not, create it

            response = requests.get("https://desi-skyportal.lbl.gov/api/candidates/{}".format(objID), headers={"Authorization": f"token {token}"})
            if response.status_code == 400:
                obj_data = {
                        "ra": ra_dec_data['TARGET_RA'][0], #RA is required when creating a new object.
                        "dec": ra_dec_data['TARGET_DEC'][0], #Same for DEC
                        "id": objID,
                        "redshift": zinfo['Z'][0],
                        "filter_ids": [sp.filter_id(filter_name)],
                        "passed_at": str(datetime.datetime.utcnow()) #UTC time when the object passed the filter
                        }

                response = requests.post(
                        "https://desi-skyportal.lbl.gov/api/candidates",
                        json= obj_data,
                        headers={"Authorization": f"token {token}"})

            #Now, we send the differenced spectrum to SkyPortal
            #First, check if the same spectrum already exists under the object
            response = requests.get("https://desi-skyportal.lbl.gov/api/sources/{}/spectra".format(objID),headers={"Authorization": f"token {token}"})
            post = True
            # for i in range (len(response.json()['data']['spectra'])):
            #     if (wavelengths == response.json()['data']['spectra'][i]['wavelengths'] and flux == response.json()['data']['spectra'][i]['fluxes']):
            #         post = False
            #Only if the exact same spectrum doesn't already exist, upload it.
            if post:
                #Send difference spectrum to SP:
                diff_spectrum_data = {
                        "obj_id": objID,
                        "wavelengths": difwave['brz'].tolist(),
                        "fluxes": difflux['brz'].tolist(),
                        "observed_at": str(night)[:4]+'-'+str(night)[4:6]+'-'+str(night)[6:]+' '+'00:00:00.000000', # Date converted into UTC time format
                        "origin": "DESIDIFF_everest", #Only the difference spectrum gets this tag in order to distinguish it on SkyPortal
                        "group_ids": [sp.group_id("DESI")],
                        "instrument_id": sp.instrument_id()
                        }
                response = requests.post(
                            '{}/api/spectrum'.format(sp.url),
                            json= diff_spectrum_data,
                            headers={"Authorization": f"token {token}"}) 

                #Send new spectra to SP:
                new_spectra_data = {
                    "obj_id": objID,
                    "wavelengths":newwave['brz'].tolist(),
                    "fluxes": newflux['brz'].tolist(),
                    "observed_at": str(night)[:4]+'-'+str(night)[4:6]+'-'+str(night)[6:]+' '+'00:00:00.000000', # Date converted into UTC time format
                    "origin": "DESIDIFF_everest",
                    "group_ids": [sp.group_id("DESI")],
                    "instrument_id": sp.instrument_id()
                    }
                response = requests.post(
                        '{}/api/spectrum'.format(sp.url),
                        json= new_spectra_data,
                        headers={"Authorization": f"token {token}"}) 

                #Code to find the ref_night that is closest to new_night to be used as 'observed_at' for SP
                new_night = datetime.datetime(int(str(night)[:4]), int(str(night)[4:6]), int(str(night)[6:]))
                closest_night = datetime.datetime(int(str(ref_nights[0])[:4]), int(str(ref_nights[0])[4:6]), int(str(ref_nights[0])[6:]))
                for ref_night in ref_nights[1:]:
                    d = datetime.datetime(int(str(ref_night)[:4]), int(str(ref_night)[4:6]), int(str(ref_night)[6:]))
                    time_delta = new_night - d
                    best = new_night - closest_night 
                    if (time_delta.total_seconds() < best.total_seconds()):
                        closest_night = d

                ref_spectra_data = {
                    "obj_id": objID,
                    "wavelengths": refwave['brz'].tolist(),
                    "fluxes": refflux['brz'].tolist(),
                    "observed_at": str(closest_night),
                    "origin": "DESIDIFF_everest",
                    "group_ids": [sp.group_id("DESI")],
                    "instrument_id": sp.instrument_id()
                    }
                response = requests.post(
                        '{}/api/spectrum'.format(sp.url),
                        json= ref_spectra_data,
                        headers={"Authorization": f"token {token}"})    

            pdf.close()
            pdf1.close()



IndentationError: unexpected indent (3235519264.py, line 43)

In [2]:
# read in and store in one place all the fibermap information in the spectra files
night = 20210608

### this needs to something other than cumulative?
for filename in glob(f"/global/project/projectdirs/desi/spectro/redux/daily/tiles/cumulative/*/{night}/spectra-*.fits"):
    print(filename)
    
    tile = filename.split('/')[-3]
    petal = filename.split('-')[1]
    current_spectra = read_spectra(filename)
    
    ### select only spectra from that night
    current_spectra = desispec.coaddition.coadd_cameras(current_spectra.select(nights = night))
    print(current_spectra.flux.keys())
    """    
    t = Table.read(filename, format='fits',hdu=1, memmap=True) 
    ##### DAVE'S ADDITION ##############
    targetcols = [i for i in t.colnames if i[-7:] =='_TARGET']
    nonzerocheck = [True in k for k in [[j != 0 for j in t[targetcols][i]] for i in range(len(t))]]
    #a really ugly line, basically generates a list of bools, 
    #True if there is at least one nonzero element in all columns ending in _TARGET
    t.remove_rows([i for i, val in enumerate(nonzerocheck) if not val])
    #This gets the index of all False values from the previous list and removes those rows
    t = t['TARGETID','TARGET_RA','TARGET_DEC','TILEID','OBJTYPE','PETAL_LOC','FIBERSTATUS','NIGHT']
    ######## END DAVE'S ADDITION ############
    t=t[numpy.logical_and(t['OBJTYPE']=='TGT', t['FIBERSTATUS']==0)]
    """

    for filename in glob(f"/global/cfs/cdirs/desi/spectro/redux/*/tiles/cumulative/{tile}/*/coadd-{petal}-{tile}*.fits"):
        print(filename)
        try:
            if filename.split('/')[-6] == 'fuji' or filename.split('/')[-6] == 'guadalupe':
                ref_spectra = desispec.coaddition.coadd_cameras(read_spectra(filename))
                print('ref_spectra from {}'.format(filename))
                norm = normalization(current_spectra.flux, current_spectra.mask, ref_spectra.flux, ref_spectra.mask)
                
                # need to instantiate a Spectra object for the difference. 
                # Using 'dif_spectra = Spectra()' is bugging on dif_spectra.mask type=NoneType, can't assign.
                dif_spectra = current_spectra
                # any problem with hardcoding in 'brz' for key in the following:
                for key in (current_spectra.flux).keys():
                    current_spectra.flux[key] = current_spectra.flux[key]/norm
                    current_spectra.ivar[key] = current_spectra.ivar[key]*norm**2  
                    dif_spectra.flux[key] = current_spectra.flux[key] - ref_spectra.flux[key]
                    # subtraction of current and reference fluxes
                    dif_spectra.flux[key] = current_spectra.flux[key] - ref_spectra.flux[key]
                    ### couldn't dif_spectra.mask[key] == 2 by summing current.mask and reference.mask
                    # summation of current and reference masks
                    dif_spectra.mask[key] = current_spectra.mask[key] + ref_spectra.mask[key]
                    # inverted summation of current and spectra inverse variance
                    ### still returning RuntimeWarning: divide by zero encountered in true_divide but not in infinite loop for the moment
                    dif_spectra.ivar[key] = 1./(1./current_spectra.ivar[key] + 1./ref_spectra.ivar[key])
                
                # mean-subtracted difference
                diff_flux_clipmean = clipmean(diff_flux,diff_ivar,diff_mask)
                
        except:
            pass    

/global/project/projectdirs/desi/spectro/redux/daily/tiles/cumulative/1097/20210608/spectra-6-1097-thru20210608.fits
INFO:spectra.py:291:read_spectra: iotime 0.724 sec to read spectra-6-1097-thru20210608.fits at 2022-06-22T14:01:48.717133
dict_keys(['brz'])
/global/cfs/cdirs/desi/spectro/redux/everest/tiles/cumulative/1097/20210608/coadd-6-1097-thru20210608.fits
/global/cfs/cdirs/desi/spectro/redux/daily/tiles/cumulative/1097/20210608/coadd-6-1097-thru20210608.fits
/global/cfs/cdirs/desi/spectro/redux/guadalupe/tiles/cumulative/1097/20210608/coadd-6-1097-thru20210608.fits
INFO:spectra.py:291:read_spectra: iotime 0.613 sec to read coadd-6-1097-thru20210608.fits at 2022-06-22T14:01:57.375698
ref_spectra from /global/cfs/cdirs/desi/spectro/redux/guadalupe/tiles/cumulative/1097/20210608/coadd-6-1097-thru20210608.fits
diff_flux done
/global/project/projectdirs/desi/spectro/redux/daily/tiles/cumulative/1097/20210608/spectra-4-1097-thru20210608.fits
INFO:spectra.py:291:read_spectra: iotime 0.

KeyboardInterrupt: 